In [21]:
import pandas as pd

In [22]:
estacion = "SUROESTE2"

d20_21 = pd.read_excel('DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx', sheet_name=estacion)
d22_23 = pd.read_excel('DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx', sheet_name=estacion)
df = pd.concat([d20_21,d22_23])

#estacion = estacion + "DROP"

In [23]:
df['date'] = pd.to_datetime(df['date'])
df["deltaDate"] = (df["date"] - df.iloc[0,0]).apply(lambda x: x.value/3.6e+12)
df.head()

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR,deltaDate
0,2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
1,2020-01-01 01:00:00,2.50,NaN,NaN,NaN,8.0,42.0,37.0,715.0,0.00,89.0,3.4,0.0,10.36,0.6,136.0,1.0
2,2020-01-01 02:00:00,2.59,NaN,NaN,NaN,10.0,43.0,42.0,714.6,0.00,89.0,3.3,0.0,10.40,0.6,111.0,2.0
3,2020-01-01 03:00:00,2.60,NaN,NaN,NaN,7.0,47.0,36.0,714.5,0.00,89.0,3.1,0.0,10.55,0.6,301.0,3.0
4,2020-01-01 04:00:00,2.71,NaN,NaN,NaN,3.0,50.0,47.0,714.2,0.01,89.0,2.9,0.0,10.71,NaN,117.0,4.0


In [24]:
print(df.shape)
df.isna().sum()

(31790, 17)


date            0
CO           1332
NO           5543
NO2          5035
NOX          5033
O3            785
PM10         1789
PM2.5        4089
PRS           346
RAINF         343
RH            469
SO2          3680
SR             44
TOUT          340
WSR          6671
WDR           835
deltaDate       0
dtype: int64

In [25]:
#df= df.dropna(subset = ["NOX", "SO2", "TOUT", "RH", "SR", "PM2.5"])
#print(df.shape)
#df.isna().sum()

In [26]:
from sklearn.impute import KNNImputer as imputer
k = imputer(n_neighbors=4)
datos = k.fit_transform((df.iloc[:, 1:]).to_numpy())


In [27]:
limpios = pd.DataFrame(datos, columns=df.columns[1:])
limpios["date"] = list(df["date"])
limpios.to_csv(f"datosCompletos{estacion}.csv")

In [28]:
intervalos={
    "SO2":[0.008, 0.11, 0.165, 0.22],
    "NO2":[8.75, 11, 13.3, 15.5],
    "etiqueta": ["bueno", "aceptable", "mala", "muy mala"]
}
def change(val, col):
    if val< intervalos[col][0]:
        return "bueno"
    if val< intervalos[col][1]:
        return "aceptable"
    if val< intervalos[col][2]:
        return "mala"
    if val< intervalos[col][3]:
        return "muy mala"
    return "Extremadamente mala"

In [29]:
datos = limpios[["date", "deltaDate","NO","NO2", "NOX", "SO2", "TOUT", "RH", "SR", "PM2.5"]]
cols_ppm = ['NO','NO2','NOX','SO2']
for col in cols_ppm:
    datos[col+'PPM'] = datos[col]*0.001
datos["SO2_discret"] = datos["SO2PPM"].apply(lambda x: change(x, "SO2"))
datos.to_csv(f"datos{estacion}.csv")

C:\Users\engel\AppData\Local\Temp\ipykernel_32052\258482312.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos[col+'PPM'] = datos[col]*0.001
C:\Users\engel\AppData\Local\Temp\ipykernel_32052\258482312.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos[col+'PPM'] = datos[col]*0.001
C:\Users\engel\AppData\Local\Temp\ipykernel_32052\258482312.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [30]:
datos.head()

,date,deltaDate,NO,NO2,NOX,SO2,TOUT,RH,SR,PM2.5,NOPPM,NO2PPM,NOXPPM,SO2PPM,SO2_discret
0,2020-01-01 00:00:00,0.0,1.725,2.250,1.150,3.175,10.505,89.0,0.0,40.5,0.001725,0.002250,0.001150,0.003175,bueno
1,2020-01-01 01:00:00,1.0,1.725,3.025,1.375,3.400,10.360,89.0,0.0,37.0,0.001725,0.003025,0.001375,0.003400,bueno
2,2020-01-01 02:00:00,2.0,1.725,3.025,1.375,3.300,10.400,89.0,0.0,42.0,0.001725,0.003025,0.001375,0.003300,bueno
3,2020-01-01 03:00:00,3.0,1.550,2.250,1.150,3.100,10.550,89.0,0.0,36.0,0.001550,0.002250,0.001150,0.003100,bueno
4,2020-01-01 04:00:00,4.0,1.725,3.025,1.375,2.900,10.710,89.0,0.0,47.0,0.001725,0.003025,0.001375,0.002900,bueno


In [31]:
datos["SO2_discret"] = datos["SO2PPM"].apply(lambda x: change(x, "SO2"))

#limpios["NO2_discret"] = limpios["NO2"].apply(lambda x: change(x, "NO2"))
#limpios["SO2_discret"] = (limpios["SO2"]*0.001).apply(lambda x: change(x, "SO2"))

C:\Users\engel\AppData\Local\Temp\ipykernel_32052\3441339858.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos["SO2_discret"] = datos["SO2PPM"].apply(lambda x: change(x, "SO2"))


In [32]:
#limpios["SO2_pm"] = limpios["SO2"]*0.001

In [33]:
#limpios.head()

In [34]:
datos = limpios[["date", "deltaDate","NO","NO2", "NOX", "SO2", "TOUT", "RH", "SR",	"SO2_discret",	"SO2_pm"]]

KeyError: "['SO2_discret', 'SO2_pm'] not in index"

In [ ]:
datos.head()

,date,deltaDate,NO,NO2,NOX,SO2,TOUT,RH,SR,NO2_discret,SO2_discret,SO2_pm
0,2020-01-01 00:00:00,0.0,4.3,6.375,10.625,0.850,12.2675,92.75,0.000,bueno,bueno,0.000850
1,2020-01-01 01:00:00,1.0,2.0,5.200,7.200,1.775,11.6800,93.00,0.037,bueno,bueno,0.001775
2,2020-01-01 02:00:00,2.0,2.6,5.000,7.600,0.750,11.6800,93.00,0.037,bueno,bueno,0.000750
3,2020-01-01 03:00:00,3.0,3.1,5.900,8.900,0.725,11.7000,93.00,0.037,bueno,bueno,0.000725
4,2020-01-01 04:00:00,4.0,2.0,4.500,6.500,0.775,11.8000,93.00,0.037,bueno,bueno,0.000775


In [ ]:
#datos.to_csv("limpios.csv")